# Imports and Data

In [1]:
!pip install -q torchvision
!pip install -q pytorch_lightning
!pip install -q torchmetrics

In [3]:
import numpy as np
import random
import os
from PIL import Image as im
import torch
import torch.nn as nn
from torch.optim import Adam
from torchmetrics import Accuracy
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def set_seed(worker_ID = 0):
  # set seed.
  worker_seed = (torch.initial_seed() + worker_ID) % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

def load_data(data_dir):
  # load data
  files = os.listdir(data_dir)
  num_data = len(files)

  # initialize data array by finding dimensions
  # MUST DOWNSAMPLE ARRAY otherwise too much info
  image_PATH = data_dir + '/' + files[0]
  img = im.open(image_PATH)
  img_arr = np.asarray(img)
  sz = len(np.ravel(img_arr))
  X = np.zeros((num_data, sz))

  # store data in array
  for i in np.arange(num_data):
      image_PATH = data_dir + '/' + files[i]
      img = im.open(image_PATH)
      img_arr = np.asarray(img)
      X[i] = np.ravel(img_arr)



In [5]:
image_dir_PATH = 'D:/DATASET-1/Dataset-1_high_resolution'


In [16]:
files = os.listdir(image_dir_PATH)
num_data = 20 #len(data)

# initialize data array by finding dimensions
image_PATH = image_dir_PATH + '/' + files[0]
img = im.open(image_PATH)
img_arr = np.asarray(img)
sz = len(np.ravel(img_arr))
X = np.zeros((num_data, sz))

# store data in array
for i in np.arange(num_data):
    image_PATH = image_dir_PATH + '/' + files[i]
    img = im.open(image_PATH)
    img_arr = np.asarray(img)
    X[i] = np.ravel(img_arr)


(20, 53747712)
